In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import Calendar
from sqlalchemy import create_engine, text

# Configuração de conexão com o banco de dados
DB_USER = 'postgres'
DB_PASSWORD = '1010'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'DB_Final'

class AgendamentoApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Agendar Consulta")

        # Conectar ao banco de dados
        self.conn = self.connect_db()

        # Criar interface
        self.criar_interface()

    def connect_db(self):
        try:
            conexao_bd = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
            engine = create_engine(
                conexao_bd,
                connect_args={'options': '-csearch_path=sistema,public'}
            )
            conn = engine.connect()
            return conn
        except Exception as e:
            print(f"Erro ao conectar ao banco de dados: {e}")
            return None

    def buscar_medicos(self):
        try:
            medico_query = text("SELECT ID_Medico, CRM, Nome, Especializacao FROM Medico JOIN Pessoa ON Medico.ID_Pessoa = Pessoa.ID_Pessoa")
            return self.conn.execute(medico_query).fetchall()
        except Exception as e:
            print(f"Erro ao buscar médicos: {str(e)}")
            return []

    def buscar_clientes(self):
        try:
            cliente_query = text("SELECT ID_Cliente, Nome, CPF FROM Cliente JOIN Pessoa ON Cliente.ID_Pessoa = Pessoa.ID_Pessoa")
            return self.conn.execute(cliente_query).fetchall()
        except Exception as e:
            print(f"Erro ao buscar clientes: {str(e)}")
            return []

    def buscar_secretarios(self):
        try:
            secretario_query = text("SELECT ID_Secretario, Nome FROM Secretario JOIN Pessoa ON Secretario.ID_Pessoa = Pessoa.ID_Pessoa")
            return self.conn.execute(secretario_query).fetchall()
        except Exception as e:
            print(f"Erro ao buscar secretários: {str(e)}")
            return []

    def criar_interface(self):
        # Secretário
        tk.Label(self.root, text="Secretário").grid(row=0, column=0)
        self.combo_secretario = ttk.Combobox(self.root, width=50, state="readonly")
        self.combo_secretario.grid(row=0, column=1)
        secretarios = self.buscar_secretarios()
        self.combo_secretario['values'] = [f"{secretario[1]}" for secretario in secretarios]
        if secretarios:
            self.combo_secretario.current(0)

        # Médico
        tk.Label(self.root, text="Médico").grid(row=1, column=0)
        self.combo_medico = ttk.Combobox(self.root, width=50, state="readonly")
        self.combo_medico.grid(row=1, column=1)
        medicos = self.buscar_medicos()
        self.combo_medico['values'] = [f"CRM: {medico[1]} - {medico[2]} ({medico[3]})" for medico in medicos]
        if medicos:
            self.combo_medico.current(0)

        # Cliente
        tk.Label(self.root, text="Cliente").grid(row=2, column=0)
        self.combo_cliente = ttk.Combobox(self.root, width=50, state="readonly")
        self.combo_cliente.grid(row=2, column=1)
        clientes = self.buscar_clientes()
        self.combo_cliente['values'] = [f"{cliente[1]} (CPF: {cliente[2]})" for cliente in clientes]
        if clientes:
            self.combo_cliente.current(0)

        # Calendário
        tk.Label(self.root, text="Data").grid(row=3, column=0)
        self.calendario = Calendar(self.root, selectmode="day", date_pattern="yyyy-mm-dd")
        self.calendario.grid(row=3, column=1)

        # Horário (Horas e Minutos)
        tk.Label(self.root, text="Horário").grid(row=4, column=0)
        self.combo_hora = ttk.Combobox(self.root, values=[f"{i:02d}" for i in range(0, 24)], width=5, state="readonly")
        self.combo_hora.grid(row=4, column=1, sticky="w")
        self.combo_minuto = ttk.Combobox(self.root, values=[f"{i:02d}" for i in range(0, 60)], width=5, state="readonly")
        self.combo_minuto.grid(row=4, column=1, sticky="e")
        self.combo_hora.current(0)
        self.combo_minuto.current(0)

        # Observação
        tk.Label(self.root, text="Observação").grid(row=5, column=0)
        self.entry_observacao = tk.Entry(self.root, width=50)
        self.entry_observacao.grid(row=5, column=1)

        # Valor
        tk.Label(self.root, text="Valor").grid(row=6, column=0)
        self.entry_valor = tk.Entry(self.root, width=20)
        self.entry_valor.grid(row=6, column=1)

        # Botões
        btn_confirmar = tk.Button(self.root, text="Confirmar", command=self.confirmar_agendamento)
        btn_confirmar.grid(row=7, column=0)
        btn_cancelar = tk.Button(self.root, text="Cancelar", command=self.root.quit)
        btn_cancelar.grid(row=7, column=1)

    def verificar_disponibilidade_medico(self, medico_id, data, horario):
        try:
            # Verifica se já existe um agendamento no mesmo horário para o médico
            conflito_query = text("""
                SELECT COUNT(*) 
                FROM Agendamento 
                WHERE ID_Medico = :id_medico 
                AND Data = :data 
                AND Hora = :horario
            """)
            resultado = self.conn.execute(conflito_query, {
                "id_medico": medico_id,
                "data": data,
                "horario": horario
            }).fetchone()

            if resultado[0] > 0:
                return False  # Médico está ocupado nesse horário

            # Verificar se o horário está dentro do intervalo de atendimento
            horario_atendimento_query = text("""
                SELECT Hora_Inicio_Manha, Hora_Fim_Manha, 
                       Hora_Inicio_Tarde, Hora_Fim_Tarde, 
                       Hora_Inicio_Noite, Hora_Fim_Noite
                FROM Horario_Atendimento_Medico
                WHERE ID_Medico = :id_medico
                AND ID_Dia_Semana = EXTRACT(DOW FROM :data)::int + 1
            """)
            horarios_atendimento = self.conn.execute(horario_atendimento_query, {
                "id_medico": medico_id,
                "data": data
            }).fetchone()

            # Se o médico não atende nesse dia da semana
            if not horarios_atendimento:
                return False  # Médico não está disponível nesse dia

            # Extrair o horário de início e fim
            hora_inicio_manha, hora_fim_manha, hora_inicio_tarde, hora_fim_tarde, hora_inicio_noite, hora_fim_noite = horarios_atendimento

            # Verifica se o horário selecionado está dentro dos horários de atendimento
            hora, minuto = map(int, horario.split(":"))
            if hora_inicio_manha and hora_fim_manha and (hora_inicio_manha <= f"{hora}:{minuto}" <= hora_fim_manha):
                return True
            if hora_inicio_tarde and hora_fim_tarde and (hora_inicio_tarde <= f"{hora}:{minuto}" <= hora_fim_tarde):
                return True
            if hora_inicio_noite and hora_fim_noite and (hora_inicio_noite <= f"{hora}:{minuto}" <= hora_fim_noite):
                return True

            return False  # Fora do horário de atendimento
        except Exception as e:
            print(f"Erro ao verificar disponibilidade do médico: {e}")
            return False

    def confirmar_agendamento(self):
        secretario = self.combo_secretario.get()
        medico = self.combo_medico.get()
        cliente = self.combo_cliente.get()
        data = self.calendario.get_date()
        hora = self.combo_hora.get()
        minuto = self.combo_minuto.get()
        observacao = self.entry_observacao.get()
        valor = self.entry_valor.get()

        if not secretario or not medico or not cliente or not data:
            messagebox.showerror("Erro", "Todos os campos devem ser preenchidos.")
            return

        horario = f"{hora}:{minuto}"

        # Obter os IDs do secretário, médico e cliente
        secretario_id = self.buscar_secretarios()[self.combo_secretario.current()][0]
        medico_id = self.buscar_medicos()[self.combo_medico.current()][0]
        cliente_id = self.buscar_clientes()[self.combo_cliente.current()][0]

        # Verificar a disponibilidade do médico
        if not self.verificar_disponibilidade_medico(medico_id, data, horario):
            messagebox.showerror("Erro", "O médico não está disponível nesse horário.")
            return

        # Inserir o agendamento no banco de dados
        try:
            agendamento_query = text("""
                INSERT INTO Agendamento (Data, Hora, Status, Valor, Observacao, ID_Cliente, ID_Secretario, ID_Medico)
                VALUES (:data, :hora, 'Agendado', :valor, :observacao, :id_cliente, :id_secretario, :id_medico)
            """)
            self.conn.execute(agendamento_query, {
                "data": data,
                "hora": horario,
                "valor": float(valor) if valor else 0.0,
                "observacao": observacao,
                "id_cliente": cliente_id,
                "id_secretario": secretario_id,
                "id_medico": medico_id
            })
            messagebox.showinfo("Sucesso", "Agendamento confirmado!")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao salvar agendamento: {e}")
        finally:
            self.conn.close()


# Inicialização da interface gráfica
root = tk.Tk()
app = AgendamentoApp(root)
root.mainloop()


Erro ao verificar disponibilidade do médico: (psycopg2.errors.AmbiguousFunction) ERRO:  a função pg_catalog.extract(unknown, unknown) não é única
LINE 7:                 AND ID_Dia_Semana = EXTRACT(DOW FROM '2024-0...
                                            ^
HINT:  Não foi possível escolher uma função que se enquadre melhor. Você precisa adicionar conversões de tipo explícitas.

[SQL: 
                SELECT Hora_Inicio_Manha, Hora_Fim_Manha, 
                       Hora_Inicio_Tarde, Hora_Fim_Tarde, 
                       Hora_Inicio_Noite, Hora_Fim_Noite
                FROM Horario_Atendimento_Medico
                WHERE ID_Medico = %(id_medico)s
                AND ID_Dia_Semana = EXTRACT(DOW FROM %(data)s)::int + 1
            ]
[parameters: {'id_medico': 3, 'data': '2024-09-21'}]
(Background on this error at: https://sqlalche.me/e/20/f405)
Erro ao buscar secretários: (psycopg2.errors.InFailedSqlTransaction) ERRO:  transação atual foi interrompida, comandos ignorados até o 

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\jarbe\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\jarbe\AppData\Local\Temp\ipykernel_12312\1783573276.py", line 190, in confirmar_agendamento
    secretario_id = self.buscar_secretarios()[self.combo_secretario.current()][0]
                    ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
IndexError: list index out of range


Erro ao buscar secretários: (psycopg2.errors.InFailedSqlTransaction) ERRO:  transação atual foi interrompida, comandos ignorados até o fim do bloco de transação

[SQL: SELECT ID_Secretario, Nome FROM Secretario JOIN Pessoa ON Secretario.ID_Pessoa = Pessoa.ID_Pessoa]
(Background on this error at: https://sqlalche.me/e/20/2j85)


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\jarbe\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\jarbe\AppData\Local\Temp\ipykernel_12312\1783573276.py", line 190, in confirmar_agendamento
    secretario_id = self.buscar_secretarios()[self.combo_secretario.current()][0]
                    ~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
IndexError: list index out of range
